In [4]:
import tkinter as tk
from tkinter import messagebox
import random

class Minesweeper:
    def __init__(self, root):
        self.root = root
        self.root.title("Minesweeper")
        
        # Game settings
        self.size = 8  # 8x8 grid
        self.mines = 10
        
        # Create buttons
        self.buttons = []
        for row in range(self.size):
            button_row = []
            for col in range(self.size):
                btn = tk.Button(root, text=' ', width=3, height=1,
                              command=lambda r=row, c=col: self.click(r, c))
                btn.bind("<Button-3>", lambda e, r=row, c=col: self.flag(r, c))
                btn.grid(row=row, column=col)
                button_row.append(btn)
            self.buttons.append(button_row)
        
        # Setup game
        self.setup_game()
        
        # Add reset button
        tk.Button(root, text="Reset", command=self.setup_game).grid(row=self.size, columnspan=self.size)
    
    def setup_game(self):
        """Initialize a new game"""
        # Create empty board
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.revealed = [[False for _ in range(self.size)] for _ in range(self.size)]
        self.flags = [[False for _ in range(self.size)] for _ in range(self.size)]
        self.game_over = False
        
        # Place mines randomly
        mines_placed = 0
        while mines_placed < self.mines:
            row = random.randint(0, self.size-1)
            col = random.randint(0, self.size-1)
            if self.board[row][col] != -1:
                self.board[row][col] = -1  # -1 represents a mine
                mines_placed += 1
        
        # Calculate numbers
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] != -1:
                    # Count adjacent mines
                    count = 0
                    for r in range(max(0, row-1), min(self.size, row+2)):
                        for c in range(max(0, col-1), min(self.size, col+2)):
                            if self.board[r][c] == -1:
                                count += 1
                    self.board[row][col] = count
        
        # Reset buttons
        for row in range(self.size):
            for col in range(self.size):
                self.buttons[row][col].config(text=' ', bg='lightgray', state=tk.NORMAL)
    
    def click(self, row, col):
        """Handle left click (reveal cell)"""
        if self.game_over or self.revealed[row][col] or self.flags[row][col]:
            return
        
        self.revealed[row][col] = True
        self.buttons[row][col].config(relief=tk.SUNKEN, state=tk.DISABLED)
        
        if self.board[row][col] == -1:  # Mine clicked
            self.game_over = True
            self.show_all_mines()
            messagebox.showinfo("Game Over", "You hit a mine!")
            return
        elif self.board[row][col] > 0:  # Number cell
            self.buttons[row][col].config(text=str(self.board[row][col]))
        else:  # Empty cell
            self.buttons[row][col].config(text=' ')
            # Reveal adjacent cells
            for r in range(max(0, row-1), min(self.size, row+2)):
                for c in range(max(0, col-1), min(self.size, col+2)):
                    self.click(r, c)
        
        # Check for win
        if self.check_win():
            self.game_over = True
            messagebox.showinfo("Congratulations!", "You won!")
    
    def flag(self, row, col):
        """Handle right click (place flag)"""
        if self.game_over or self.revealed[row][col]:
            return
        
        self.flags[row][col] = not self.flags[row][col]
        if self.flags[row][col]:
            self.buttons[row][col].config(text='🚩', bg='orange')
        else:
            self.buttons[row][col].config(text=' ', bg='lightgray')
    
    def show_all_mines(self):
        """Show all mines when game is lost"""
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] == -1:
                    self.buttons[row][col].config(text='💣', bg='red')
    
    def check_win(self):
        """Check if player has won"""
        for row in range(self.size):
            for col in range(self.size):
                if self.board[row][col] != -1 and not self.revealed[row][col]:
                    return False
        return True

# Create and run the game
root = tk.Tk()
game = Minesweeper(root)
root.mainloop()